## 高斯消元法or BP 主要是解码算法，或者未来使用的那些升级算法。 主要参考 主要是异或方程组
## 一定要注意这个是异或方程组。
下面对应说明
增广矩阵，A*X=Y,,,      
A对应编码矩阵，n*k，生成n个编码行向量。
X是k个原始行向量。
进行的运算是异或运算。

### 仿真结构
1. source_code
包含产生的原始数据，信息长度length_bytes(8*length)，原始信息的个数k 
使用vscode里面的hextemp可以查看

2. 

In [130]:
ss=source_code(7,1,"raptor_test")
with open("raptor_a","wb") as fd:
    now=int("00100100",2)
    fd.write(struct.pack("B",now))
    now=int("11110100",2)
    fd.write(struct.pack("B",now))
    now=int("00101010",2)
    fd.write(struct.pack("B",now))
    now=int("10111110",2)
    fd.write(struct.pack("B",now))
    now=int("10111000",2)
    fd.write(struct.pack("B",now))
    now=int("01010110",2)
    fd.write(struct.pack("B",now))
    now=int("00100000",2)
    fd.write(struct.pack("B",now))
    

ee=encoder(7,"raptor_test",1,"raptor_encoder")
ee.load_A_encoding("raptor_a")


(7, 1)
(7, 1)
7


In [10]:
ddd=decoder(7,1,"raptor_encoderload_A",file_name="decoder_data_raptor",source_data_file="raptor_test")


ddd.inactivation()




7
start of self.encoder [[162]
 [ 62]
 [155]
 [235]
 [ 77]
 [  6]
 [ 54]]
 self.encoder [[162]
 [ 62]
 [155]
 [235]
 [ 77]
 [  6]
 [ 54]]
this step is recoverable
the recover is 2
the encoed is encoder 6
 self.encoder [[148]
 [  8]
 [173]
 [221]
 [123]
 [  6]
 [ 54]]
this step is recoverable
the recover is 5
the encoed is encoder 0
 self.encoder [[148]
 [156]
 [173]
 [ 73]
 [123]
 [146]
 [ 54]]
this step is inactive 
the inactive is the source 4
the encoed is encoder 2
 self.encoder [[148]
 [156]
 [173]
 [ 73]
 [123]
 [146]
 [ 54]]
this step is recoverable
the recover is 6
the encoed is encoder 2
 self.encoder [[148]
 [156]
 [173]
 [228]
 [123]
 [ 63]
 [ 54]]
this step is inactive 
the inactive is the source 0
the encoed is encoder 3
 self.encoder [[148]
 [156]
 [173]
 [228]
 [123]
 [ 63]
 [ 54]]
this step is recoverable
the recover is 3
the encoed is encoder 3
 self.encoder [[148]
 [120]
 [173]
 [228]
 [159]
 [219]
 [ 54]]
this step is recoverable
the recover is 1
the encoed is encode

In [135]:
hex(214)

'0xd6'

In [25]:
import random
import struct
import numpy as np
from math import *
from collections import defaultdict
class source_code:

    def __init__(self,k,length_bytes,file_name="source_code"):
        self.k=k
        self.length_bytes=length_bytes
        self.file_name=file_name
        self.__generate_random_source_code()

    def __generate_random_source_code(self):
        '''
        generate k*8Llength_bytes bits bin_file
        @parameter k :numer of source code 
        @parmamter file_name="source_code" by default
        @parmeter length_bytes 8*length_bits
        
        '''
        k=self.k
        length_bytes=self.length_bytes
        file_name=self.file_name
        code_list=[struct.pack("B",i)  for i in range(256)]

        with open(file_name,"wb") as fd:
            for i in range(k*length_bytes):
                fd.write(code_list[random.randint(0,255)])
class encoder:
    def __init__(self,k,source_code,length_bytes,file_name="encoding_data"):
        self.k=k
        self.file_name=file_name
        self.source_code=source_code
        self.length_bytes=length_bytes
        self.read_source()
    def read_source(self,):
        source_code=self.source_code
        length_bytes=self.length_bytes
        k=self.k
        #self.data=np.zeros((k,length_bytes),dtype=int)
        with open(source_code,"rb") as fd:
            one_source_code=fd.read(length_bytes)
            self.data=np.array(list(one_source_code),dtype="u1").reshape(1,length_bytes)
            count=1
            while(1):
                one_source_code=fd.read(length_bytes)
                if not one_source_code:
                    break
                new_line=np.array(list(one_source_code),dtype="u1").reshape(1,length_bytes)
                self.data=np.concatenate((self.data,new_line))
                count+=1
        if count!=self.k:
            raise Exception("the source code not match the size")

        
        pass
    def load_A_encoding(self,A_name):
        file_name=self.file_name+"load_A"
        A=self.__encoding_load_A(A_name)
        self.__encoding(A,file_name)
        print(A.shape)
        print(self.data.shape)
    def __encoding_load_A(self,A_name):
        k=self.k
        A=None
        length=ceil(k/8)
        with open(A_name,"rb") as fd:
            while(1):
                new_line=list(fd.read(length))
                if not(new_line):
                    break
                if A is None:
                    A=np.array(new_line,dtype="u1").reshape(1,length)
                else:
                    A=np.concatenate((A,np.array(new_line,dtype="u1").reshape(1,length)),axis=0)
            
            
        return A

        pass
    def random_encode(self,n):
        file_name=self.file_name+"random"
        A=self.__random_A(n)
        self.__encoding(A,file_name)
        print(A.shape)
        print(self.data.shape)
        pass
    def __random_A(self,n):
        k=self.k
        A=np.random.randint(0,256,(n,ceil(k/8)),dtype="u1")
        return A
        pass
    def __encoding(self,A,file_name):
        '''
        AX=Y, A:(n,k)  X(k,length_bytes) Y(n,length_bytes)
        这里有点麻烦，因为还要写入编码矩阵。因为要按照bit写入，所以应该写入ceil(n,8),最后一个，的后
        几位是没有用到，但是写这里。
        '''
        k=self.k
        n=A.shape[0]
        length_bytes=self.length_bytes
        with open(file_name,"wb") as fd:
            for i in range(n):
                new_line=np.zeros((length_bytes),dtype="u1")
                number=0
                for j in range(ceil(k/8)):
                    esi_bytes=A[i][j]
                    fd.write(struct.pack("B",esi_bytes))
                    for v in range(8):
                        if number>=k:
                            break
                        if esi_bytes&(1<<7):
                            new_line=new_line^self.data[number]
                        esi_bytes=esi_bytes<<1
                        number+=1
                new_line=[struct.pack("B",bytes_newline)for bytes_newline in new_line]
                #print(new_line)
                fd.write(b''.join(new_line))
            



        
        
       

    def rs_encode(self):

        file_name=self.file_name+"rs"
        pass
class bec_channel:
    def __init__(self,loss,seed=10):
        self.loss=loss
        
        random.seed(seed)
    def transfer(self,k,length_bytes,encoder_data_file,file_name="after_bec_encoder",):
        
        
        length=length_bytes+ceil(k/8)
        threshold=10000*self.loss
        '''
        this is the file name 
        '''
       # print(encoder_data_file)
        with open(encoder_data_file,"rb") as fd1, open(file_name,"wb") as fd2:
            while(1):
                message=fd1.read(length)
                if not message:
                    break
                success=(random.randint(0,10000)>threshold)
                if success:
                    fd2.write(message)

class decoder:
    def __init__(self,k,length_bytes,bec_encode_data_file,file_name="decoder_data",source_data_file=None):
        self.file_name=file_name
        self.k=k
        
        self.bec_encode_data_file=bec_encode_data_file
        self.source_data_file=source_data_file
        self.length_bytes=length_bytes
        self.length_encoder=ceil(k/8)+length_bytes
        self.__read_encoder_data()
        self.decoder_data=np.zeros((k,length_bytes),dtype="u1")
        
    def __read_encoder_data(self,):
        '''
        从encoder data 中提取两个变量一个是A，一个是Y
        '''
        k=self.k
        length_bytes=self.length_bytes
        head=ceil(k/8)
        length=head+length_bytes
        A=None
        source_data=None
        with open(self.bec_encode_data_file,"rb") as fd1:
            while(1):
                message=fd1.read(length)
                if not message:
                    break
                message=list(message)
                if A is None:
                    A=np.array(message[:head],dtype="u1").reshape(1,head)
                    source_data=np.array(message[head:],dtype="u1").reshape(1,length_bytes)
                    continue
                else:
                    new_line_A=np.array(message[:head],dtype="u1").reshape(1,head)
                    new_line_source_data=np.array(message[head:],dtype="u1").reshape(1,length_bytes)
                    A=np.concatenate((A,new_line_A),axis=0)
                    source_data=np.concatenate((source_data,new_line_source_data),axis=0)
        
        self.A=A
        self.encoder_data=source_data
        print(A.shape[0])
    def __find_first_1(self,pos_i,):
        success=0
        pos=(pos_i//8,pos_i%8)
        for i in range(pos_i,self.A.shape[0]):
            
            if self.A[i][pos[0]]&(1<<(7-pos[1]))==0:
                continue
            else:
                success=1
                break
        if not success:
            return None
        else:
            #print(f"__find_first_1,pos_i{pos_i},i{i}")
            self.A[[pos_i,i],:]=self.A[[i,pos_i],:]
            
            self.encoder_data[[pos_i,i],:]=self.encoder_data[[i,pos_i],:]
            #print(self.A)
            return 1

    def __eliment_1_in_other_line(self,pos_i):
        pos=(pos_i//8,pos_i%8)
        for i in range(self.A.shape[0]):
            if i==pos_i:
                continue
            if self.A[i][pos[0]]&(1<<(7-pos[1]))==0:
                continue
            else:
                self.A[i]=self.A[i]^self.A[pos_i]
                self.encoder_data[i]=self.encoder_data[i]^self.encoder_data[pos_i]
    def __set_add(self,a,b):
        c=set.union(set.difference(a,b),set.difference(b,a))
        return c
    def inactivation(self,):
        file_name=self.file_name+"inactive"
        '''
         参考raptor 文章里面提供的失活译码算法。
         先进行bp译码算法，尽可能的消元，消元完的，需要移位，然后放置到前面
         然后找不到度为1的时候，采用inactivation，然后再找
         这里这个如何选择inactivation的逻辑是一个优化问题，应该找degree-2-chanin最长的那个，可是这里面因为所以我就选择degree最小的随便的一个就好了。然后标记，然后再做，这个真的不是很好表达。
        最后把所有的inactive这写矩阵，矩阵，采用高斯消元实现。
        然后再反代回去。
         
         完成需要的逻辑应该有点类似于，所以我需要逻辑矩阵，然后
        并不是像bp译码算法那样，完全从图论 的角度来理解，我使用矩阵的角度来实现
        
        这个需要矩阵和使用度和所以这里要使用两个方法的和要不度的寻找就要通过遍历的方法，也不方便
        要存2个矩阵，一个是degree这个图，一个是y，还有对应的序号矩阵。

        '''
        ###这里和高斯算法不太一样的点在于如何找度为1的点。因为找度为1的点是有难度的。
        ###我决定使用图论的方法来实现
        ###1. 首先每个度都要找到对应的集合点，这是一个映射集合degree2source
        ###2. encoder2source
        ###3. source2encoder
        ###4. encoder2补偿
        ### 过程应该是去努力找度为1的点，没有就inactive，然后使后继续。
        ### 直到回复了有限的x，剩下的所有y去解矩阵。这里剩下的y，代表的是没有用来表达x的向量。
        totol_source_number=self.k
        totol_encoder_number=self.A.shape[0]
        head=ceil(self.k/8)
        dic_degree2encode=defaultdict(set)###degree 是encoder的degree
        dic_encode2source=defaultdict(set)
        dic_source2encode=defaultdict(set)
        dic_encoder2com=defaultdict(set)##这个初始话已经完成了
        decoder_data_patent=[]
        '''
        初始化3个dic，然后开始bp姨妈算法
        '''
        k=self.k
        for encoder_number in range(totol_encoder_number):
            count=0
            for head_bytes in range(head):
                key=self.A[encoder_number][head_bytes]
                
                for pos in range(8):
                    source_number=pos+head_bytes*8
                    if source_number>=k:
                        break
                    if key&(1<<(7-pos)):
                        dic_encode2source[encoder_number].add(source_number)
                        dic_source2encode[source_number].add(encoder_number)
                        count+=1
                    else:
                        continue
            dic_degree2encode[count].add(encoder_number)
        #print(dic_degree2encode)
        #print(dic_encode2source)
        #print(dic_source2encode)
        source_recover_number=0
        inactive_list=[]
        #print("start of self.encoder",self.encoder_data)


        while(1):

            #print(" self.encoder",self.encoder_data)
            inactive=0
            if source_recover_number==totol_source_number:
                
                print("step1 finished")
                print("next start the gas_eli")
                print(f"inactive_number is {len(inactive_list)}")
                
                break 
            if len(dic_degree2encode[1])==0:
                
                '''
                下面开始失活的过程
                选择度最小的那个encoder向量的一个使用

                '''
                inactive=1
                minn_degree=2
                while(1):
                    if len(dic_degree2encode[minn_degree])==0:
                        minn_degree+=1
                    else:
                        break
                
                
                
            if inactive==1:
                
                now_degree_one=dic_degree2encode[minn_degree].pop()
                
                dic_degree2encode[minn_degree-1].add(now_degree_one) 
                source_number_new=dic_encode2source[now_degree_one].pop()
                inactive_list.append(source_number_new)
                dic_encoder2com[now_degree_one].add(source_number_new)
                ####这里需要更新，degree，把所有包含这个source的全部删掉。
                for encoder_number_connect_source_new in dic_source2encode[source_number_new]:
                    if encoder_number_connect_source_new==now_degree_one:
                        continue
                    else:
                        '''
                        dic_degree2encode 减少度
                        dic_encode2source 删掉边
                        '''
                        
                        dic_encode2source[encoder_number_connect_source_new].remove(source_number_new)
                        ll=len(dic_encode2source[encoder_number_connect_source_new])
                        dic_degree2encode[ll].add(encoder_number_connect_source_new)
                        dic_degree2encode[ll+1].remove(encoder_number_connect_source_new)
                        dic_encoder2com[encoder_number_connect_source_new]=self.__set_add(dic_encoder2com[encoder_number_connect_source_new],dic_encoder2com[now_degree_one])
                
                source_recover_number+=1
                #print("this step is inactive ")
                #print(f"the inactive is the source {source_number_new}")
                #print(f"the encoed is encoder {now_degree_one}")
            else:

                now_degree_one=dic_degree2encode[1].pop()
            
                source_number_new=dic_encode2source[now_degree_one].pop()
                source_recover_number+=1
            #print(now_degree_one,source_number_new)
                decoder_data_patent.append((source_number_new,now_degree_one))#now_degree_one 是encoder的number
                #print("this step is recoverable")
                #print(f"the recover is {source_number_new}")
                #print(f"the encoed is encoder {now_degree_one}")
                for encoder_number_connect_source_new in dic_source2encode[source_number_new]:
                    if encoder_number_connect_source_new==now_degree_one:
                        continue
                    else:
                        '''
                        dic_degree2encode 减少度
                        dic_encode2source 删掉边
                        '''
                        self.encoder_data[encoder_number_connect_source_new]=self.encoder_data[encoder_number_connect_source_new]^self.encoder_data[now_degree_one]
                        dic_encode2source[encoder_number_connect_source_new].remove(source_number_new)
                        ll=len(dic_encode2source[encoder_number_connect_source_new])
                        dic_degree2encode[ll].add(encoder_number_connect_source_new)
                        dic_degree2encode[ll+1].remove(encoder_number_connect_source_new)
                        dic_encoder2com[encoder_number_connect_source_new]=self.__set_add(dic_encoder2com[encoder_number_connect_source_new],dic_encoder2com[now_degree_one])

            #####step2 解码
        print(inactive_list)
        #print(f"dic_encoder2com{dic_encoder2com}")
        #print(f"dic_degree2encode{dic_degree2encode}")
        #print(f"decoder_data_patent{decoder_data_patent}")
        if len(inactive_list)!=0:
        ####我这做的是对的，然后使用构建gas消元的东西。首先看有几个等式，
        
            
        ###构建相应的matrix然后再反过来
        ### 下面尝试用来构建类似于高斯消元法的需要的信息。
        ### A矩阵
        ### Data矩阵
        ### 对应关系
            success=self.__gas_els(inactive_list,dic_encoder2com,dic_degree2encode,decoder_data_patent,dic_encode2source,dic_source2encode)
            if success==0:
                print("not enough data ,decoder failed")
                return 
        ####################根据patent 来还原像一个的source data 
        


        
        
        for source_number,encoder_number in decoder_data_patent:
            self.decoder_data[source_number]=self.encoder_data[encoder_number]
            for source_com_number in dic_encoder2com[encoder_number]:
                
                self.decoder_data[source_number]=self.decoder_data[source_number]^self.decoder_data[source_com_number]

        self.__write_encoder_data(file_name)

        print("all recover")
        print("start check")
        if self.__check(file_name):
            print("success")
        else:
            print("sth is wrong")
        return
    def __find_first_1_inactive(self,pos_i,A,encoder_data):
        success=0
        pos=(pos_i//8,pos_i%8)
        
        for i in range(pos_i,A.shape[0]):
            
            if A[i][pos[0]]&(1<<(7-pos[1]))==0:
                continue
            else:
                success=1
                break
        if not success:
            return None
        else:
            #print(f"__find_first_1,pos_i{pos_i},i{i}")
            A[[pos_i,i],:]=A[[i,pos_i],:]
            
            encoder_data[[pos_i,i],:]=encoder_data[[i,pos_i],:]
            #print(self.A)
            return 1

    def __eliment_1_in_other_line__inactive(self,pos_i,A,encoder_data):
        pos=(pos_i//8,pos_i%8)
        for i in range(A.shape[0]):
            if i==pos_i:
                continue
            if A[i][pos[0]]&(1<<(7-pos[1]))==0:
                continue
            else:
                A[i]=A[i]^A[pos_i]
                encoder_data[i]=encoder_data[i]^encoder_data[pos_i]
    def  __gas_els(self,inactive_list,dic_encoder2com,dic_degree2encode,decoder_data_patent,dic_encode2source,dic_source2encode):
        k=len(inactive_list)
        #print(f"tota number of the inactive is {k}")
        ###要构建一个映射，比如inactive_list的反映射
        
        ###根据dic_degree2encode里面0的量来估算。
        n=len(dic_degree2encode[0])
        encoder_data=None
        ###要映射所以比较麻烦
        A=np.zeros((n,ceil(k/8)),dtype="u1")
        #print(f"A.shape{A.shape}")
        for i in range(n):
            y_number=dic_degree2encode[0].pop()
            if encoder_data==None:
                encoder_data=np.array(self.encoder_data[y_number],dtype="u1").reshape(1,self.length_bytes)
            else:
                new_line=np.array(self.encoder_data[y_number],dtype="u1").reshape(1,self.length_bytes)
                encoder_data=np.concatenate((encoder_data,new_line),axis=0)
            
            for j in range(k):
                pos=(j//8,j%8)
                if inactive_list[j] in dic_encoder2com[y_number]:
                    A[i][pos[0]]=A[i][pos[0]]|1<<(7-pos[1])
        
        
        #print("inactive A is ")
        #print(A)
        #print("encoder_data")
        #print(encoder_data)
        
        
        for i in range(k):
            
            success=self.__find_first_1_inactive(i,A,encoder_data)
            #print(i)
            #print(self.A)
            if success is None:
                print("con not recover ")
                return 0
            ###因为下面这句话，所以复杂度是O(nk**2)
            self.__eliment_1_in_other_line__inactive(i,A,encoder_data)
            #print(self.A)
        for i in range(k):
            #dddd=[hex(data) for data in encoder_data[i]]
            #print(f"recover source _data {inactive_list[i]}, is {dddd}")
            self.decoder_data[inactive_list[i]]=encoder_data[i]
        
        return 1
            
                

        
        pass

        


    def gas_eli(self,):
        file_name=self.file_name+"gas"
        '''
        高斯消元，就是解放成的最基本的方法。遍历
        1. 遍历k次，
        2. 每次要确认[k][k]出为1,如果不是1,就向下找到一个1,然后swap。
        3. 然后消去所有其他的在这列的1，是所有不是光下三角,xor所有含1的位置
        4. 直到k次结束，对应的头k个信息就是需要的。如果中间出现2找不到1,结束。
        用到A，和 encoder_data
        '''
        k=self.k
        A=self.A
        n=A.shape[0]
        encoder_data=self.encoder_data
        print(self.A)
        for i in range(k):
            
            success=self.__find_first_1(i)
            #print(i)
            #print(self.A)
            if success is None:
                print("con not recover ")
                return 
            ###因为下面这句话，所以复杂度是O(nk**2)
            self.__eliment_1_in_other_line(i)
            #print(self.A)
        self.decoder_data=encoder_data[:k]
        self.__write_encoder_data(file_name)

        print("all recover")
        print("start check")
        if self.__check(file_name):
            print("success")
        else:
            print("sth is wrong")
        return

            
                

        
        pass
    def bp(self):
        '''
        需要用到图论，所以需要构建两个集合，也就是两个dict，分别代表左右两边的节点
        
        dic_source——node，和该source_node 当该source_node恢复了，
        
        dic_，挑选当前度节点为1的节点，然后可以自己创建，也可以使用别人的
        首先要有一个度的dic,实现每个度，都能找到对应的集合，encoder_序号
        然后，因为度为1,的节点里面存的是哪个原始souce序号，需要一个dic，查找这个，就知道原始信息序号有什么。相当于是encoder序号映射source序号
        然后需要一个集合，所有相应的该source序号，连接的encoder序号。
        然后对于每个encoder，剔除该source，并更新。
        所以需要三个dic，对应关系，如上。流程如上。
        然后把所有的和这个原始的都剔除掉。
        '''
        k=self.k
        file_name=self.file_name+"_bp"
        totol_source_number=self.k
        totol_encoder_number=self.A.shape[0]
        head=ceil(self.k/8)
        dic_degree2encode=defaultdict(set)###degree 是encoder的degree
        dic_encode2source=defaultdict(set)
        dic_source2encode=defaultdict(set)
        '''
        初始化3个dic，然后开始bp姨妈算法
        '''
        for encoder_number in range(totol_encoder_number):
            count=0
            for head_bytes in range(head):
                key=self.A[encoder_number][head_bytes]
                
                for pos in range(8):
                    source_number=pos+head_bytes*8
                    if source_number>=k:
                        break
                    if key&(1<<(7-pos)):
                        dic_encode2source[encoder_number].add(source_number)
                        dic_source2encode[source_number].add(encoder_number)
                        count+=1
                    else:
                        continue
            dic_degree2encode[count].add(encoder_number)
        #print(dic_degree2encode)
        #print(dic_encode2source)
        #print(dic_source2encode)
        source_recover_number=0
        while(1):
            if source_recover_number==totol_source_number:
                self.__write_encoder_data(file_name)

                print("all recover")
                print("start check")
                if self.__check(file_name):
                    print("success")
                else:
                    print("sth is wrong")
                return 
            if len(dic_degree2encode[1])==0:
                print(f"{source_recover_number/totol_source_number*100}% sysmle has been recoverd")
                print(source_recover_number)
                return 
            now_degree_one=dic_degree2encode[1].pop()
            
            source_number_new=dic_encode2source[now_degree_one].pop()
            source_recover_number+=1
            #print(now_degree_one,source_number_new)
            self.decoder_data[source_number_new]=self.encoder_data[now_degree_one]
            for encoder_number_connect_source_new in dic_source2encode[source_number_new]:
                if encoder_number_connect_source_new==now_degree_one:
                    continue
                else:
                    '''
                    dic_degree2encode 减少度
                    dic_encode2source 删掉边
                    '''
                    self.encoder_data[encoder_number_connect_source_new]=self.encoder_data[encoder_number_connect_source_new]^self.encoder_data[now_degree_one]
                    dic_encode2source[encoder_number_connect_source_new].remove(source_number_new)
                    ll=len(dic_encode2source[encoder_number_connect_source_new])
                    dic_degree2encode[ll].add(encoder_number_connect_source_new)
                    dic_degree2encode[ll+1].remove(encoder_number_connect_source_new)

    def __write_encoder_data(self,file_name):
        k=self.k
        with open(file_name,"wb") as fd:
            for i in range(k):
                new_line=[struct.pack("B",bytes_source)for bytes_source in self.decoder_data[i]]
                #print(new_line)
                fd.write(b''.join(new_line))
        

        
    def __check(self,file_name):
        source_data_file=self.source_data_file
        decoder_data_file=file_name
        length=self.length_bytes
        with open(source_data_file,"rb") as fd1,open(decoder_data_file,"rb") as fd2:
            while(1):
                message_source=fd1.read(length)
                message_decoder=fd2.read(length)
                if not message_source and not message_decoder:
                    break
                elif message_decoder!=message_source:
                    return False
                else:
                    continue
        return True


        

In [12]:
## file name of the source code
source_code_file="source_code_test"
#number of source _code
k=32
##length of the code
length_bytes=3
##initial the generate the source code 
sc=source_code(k,length_bytes,source_code_file)
#####
n=100
encoder_data_file="encoder_data"
ec=encoder(k,"source_code_test",length_bytes,encoder_data_file)


In [13]:
ec.load_A_encoding("A_lt_patent")

(100, 4)
(32, 3)


In [14]:
bec=bec_channel(0.2)



In [15]:
bec.transfer(k,length_bytes,"encoder_dataload_A")

In [26]:
dec=decoder(k,length_bytes,"after_bec_encoder",file_name="decoder_data",source_data_file="source_code_test")


84


In [28]:
dec.inactivation()

step1 finished
next start the gas_eli
inactive_number is 0
[]
all recover
start check
success


In [27]:
dec.gas_eli()

[[  0   0   0   2]
 [ 73  36 146  73]
 [  0   0   8   0]
 [  2   0   1   0]
 [  0  32  32  32]
 [  0   4   0   4]
 [  0  32   1   0]
 [  0   9   0   0]
 [ 48  24  24  24]
 [  0  32   1   0]
 [  0   0 128   8]
 [168   0   1  85]
 [224   1 224   1]
 [  0   0  16   0]
 [128  64   0 128]
 [ 32   0   0   8]
 [ 32   0   2   0]
 [  8  32 146  73]
 [  0  64   0  16]
 [  0   0  64  64]
 [  8  48  96 131]
 [  0  64   0   2]
 [ 16   1   0  16]
 [ 48   0   0   0]
 [ 16   1   0 128]
 [  0   8   0   8]
 [ 32   4   0 128]
 [255 255 255 255]
 [248   0   7 255]
 [  0   0   0  12]
 [ 36  18   9  36]
 [ 34   0   0   0]
 [148 132  33   8]
 [  0  65   4   0]
 [  0   0   0   9]
 [  0   0   0 144]
 [  4   0 128  16]
 [ 75  90 214 148]
 [ 16   0   8   0]
 [  0   0   0   2]
 [  0   0  32   0]
 [192   3 224   1]
 [  2  64   0   0]
 [  0 128   0   2]
 [ 64  16   0   0]
 [  1   0   0  16]
 [  0   0  16  32]
 [ 82 148 181 173]
 [  0 128   1   0]
 [  0   6   0   0]
 [130   9  36 144]
 [  0   0   8   4]
 [  8  48  9

In [233]:
dec.A

array([[128,   0,   0,   0],
       [ 64,   0,   0,   0],
       [ 32,   0,   0,   0],
       [ 16,   0,   0,   0],
       [  8,   0,   0,   0],
       [  4,   0,   0,   0],
       [  2,   0,   0,   0],
       [  1,   0,   0,   0],
       [  0, 128,   0,   0],
       [  0,  64,   0,   0],
       [  0,  32,   0,   0],
       [  0,  16,   0,   0],
       [  0,   8,   0,   0],
       [  0,   4,   0,   0],
       [  0,   2,   0,   0],
       [  0,   1,   0,   0],
       [  0,   0, 128,   0],
       [  0,   0,  64,   0],
       [  0,   0,  32,   0],
       [  0,   0,  16,   0],
       [  0,   0,   8,   0],
       [  0,   0,   4,   0],
       [  0,   0,   2,   0],
       [  0,   0,   1,   0],
       [  0,   0,   0, 128],
       [  0,   0,   0,  64],
       [  0,   0,   0,  32],
       [  0,   0,   0,  16],
       [  0,   0,   0,   8],
       [  0,   0,   0,   4],
       [  0,   0,   0,   2],
       [  0,   0,   0,   1],
       [  0,   0,   0,   0],
       [  0,   0,   0,   0],
       [  0,  

In [6]:
a={1,2,3}
b={1,4,5}
c=set.union(set.difference(a,b),set.difference(b,a))
c

{2, 3, 4, 5}

In [203]:
dec.check()

FileNotFoundError: [Errno 2] No such file or directory: 'decoder_data'

In [124]:
from collections import defaultdict
empty=set()
a=defaultdict(set)
a[1].add(2)
a[2].add(3)
a[1].remove(2)
a[2]==empty

False

In [135]:
a=set([1,4,3])
b=a.pop()
b=a.pop()
a

{4}

In [1]:
help(set)

Help on class set in module builtins:

class set(object)
 |  set() -> new empty set object
 |  set(iterable) -> new set object
 |  
 |  Build an unordered collection of unique elements.
 |  
 |  Methods defined here:
 |  
 |  __and__(self, value, /)
 |      Return self&value.
 |  
 |  __contains__(...)
 |      x.__contains__(y) <==> y in x.
 |  
 |  __eq__(self, value, /)
 |      Return self==value.
 |  
 |  __ge__(self, value, /)
 |      Return self>=value.
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __gt__(self, value, /)
 |      Return self>value.
 |  
 |  __iand__(self, value, /)
 |      Return self&=value.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __ior__(self, value, /)
 |      Return self|=value.
 |  
 |  __isub__(self, value, /)
 |      Return self-=value.
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __ixor__(self, value, /)
 |      Re

In [1]:
'''galss elimation xor
@parameter n:encoding_vector_number|receiving_vector_number
@parameter k:the source_vector number
n>=k
@parameter Y:the receive_encoding_vector
@parameter A:the  Corresponding encoding matrix

'''


1


In [10]:
import random
import numpy as np
class int2(int):
    def __add__(self,val):
        return self^val
    def __mul__(self,val):
        return self^val
class decoding_xor:
    def __init__(self,n,k,length):
        '''
        A is corresponding with Y,when change the A,we need change the Y the same time
        @parameter self.length
        @parameter self.n
        @parameter self.k
        '''
        self.n=n
        self.k=k
        self.length=length
    def random_generate(self,):
        n=self.n
        k=self.k
        length=self.length
        A=[]
        for i in range(n):
            now=[]
            for j in range(k):
                now.append(int2(random.randint(0,1)))
            A.append(now)
        self.A=np.array(A,dtype=int2)

        Y=[]
        for i in range(n):
            now=[]
            for j in range(k):
                now.append(int2(random.randint(0,1)))
            Y.append(now)
        self.Y=np.array(Y,dtype=int2)
        print(self.Y.shape)
        self.genereate_degree()
    def genereate_degree(self,):
        self.X=np.zeros((self.k,self.length),dtype="int")
        self.Y_degree=np.array([(self.Y[i]==1).sum() for i in range(self.n)])
        self.X_finished=np.zeros((self.k),dtype="int")

    def load_A_Y(self,A,Y):
        if (self.n,self.k)!=A.shape:
            raise Exception("the size is not compatable")
        if (self.n,self.length)!=Y.shape:
            raise Exception("the size of the Y is not compatable")
        self.genereate_degree()
    
    
    def gal_eli(self):
        '''
        这个高斯消元和其他的有一点点不一样。因为元素只有1
        算法思路，遍历对角线也就是k，如果此时A[k][k]!=0,在下面的k-n里面找直到找到。
        然后对下面所有的进行行操作。相当于进行了k**2的k个行操作，所以是k**3，也可以看作k*k*（n+k）的操作
        '''
        A=self.A
        Y=self.Y
        n=self.n
        k=self.k
        ##消元，变成上三角
        for row in range(k):
            if A[row][row]!=1:
                ## swwap the first 
                for row1 in range(row+1,k):
                    if A[row1][row]==1:
                        
                        A[row1],A[row]=A[row],A[row1]
                        
                        break
                    else:
                        continue
            if A[row][row]!=1:
                continue

            ### start 移位操作
        ##回代，将上三角矩阵开始往回带。
        

                
        pass
    def bp(self,):
        ###比较简单。


### 仿真结构



In [12]:
dd.random_generate()

(10, 5)


In [16]:
dd.Y

array([[1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 0, 0],
       [0, 1, 1, 0, 0],
       [0, 1, 1, 1, 1],
       [1, 1, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [0, 1, 1, 0, 0],
       [0, 0, 1, 1, 0]], dtype=object)

0
1.0
13.0
-11.0
